In [2]:
import xgboost as xgb
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score 
from sklearn.metrics import make_scorer, mean_squared_error, log_loss
from sklearn.pipeline import make_pipeline

In [3]:
train = pd.read_csv(r"C:\Users\henry\Documents\MSBAprogram\MachineLearning1\Project Data\ProjectManipulatedTrainingData.csv")

In [4]:
train.shape

(319911, 25)

In [5]:
train.shape

(319911, 25)

In [6]:
train = train.drop(columns=['id','device_id'])

In [7]:
label_encoder = LabelEncoder()

In [8]:
train['site_id']=label_encoder.fit_transform(train['site_id'])
train['site_domain']=label_encoder.fit_transform(train['site_domain'])
train['site_category']=label_encoder.fit_transform(train['site_category'])
train['app_id']=label_encoder.fit_transform(train['app_id'])
train['app_domain']=label_encoder.fit_transform(train['app_domain'])
train['app_category']=label_encoder.fit_transform(train['app_category'])
train['device_model']=label_encoder.fit_transform(train['device_model'])
train['device_type']=label_encoder.fit_transform(train['device_type'])
train['device_conn_type']=label_encoder.fit_transform(train['device_conn_type'])
train['C14']=label_encoder.fit_transform(train['C14'])
train['C15']=label_encoder.fit_transform(train['C15'])
train['C16']=label_encoder.fit_transform(train['C16'])
train['C17']=label_encoder.fit_transform(train['C17'])
train['C19']=label_encoder.fit_transform(train['C19'])
train['C20']=label_encoder.fit_transform(train['C20'])
train['C21']=label_encoder.fit_transform(train['C21'])

In [9]:
newtrain = train.iloc[:255929]

In [10]:
newtrain.shape

(255929, 23)

In [11]:
newval = train.iloc[255929:]

In [12]:
newval.shape

(63982, 23)

In [13]:
X_train = newtrain.drop(columns='click')

In [14]:
y_train = newtrain.loc[:,'click']

In [15]:
X_test = newval.drop(columns='click')

In [16]:
y_test = newval.loc[:,'click']

In [19]:
# tuning 'learning_rate'
param_test1 = { 
 'learning_rate':[0.05,0.1,0.15,0.2,0.25,0.3]
}
gsearch = GridSearchCV(
    estimator = XGBClassifier(objective= 'binary:logistic'), 
    param_grid = param_test1, 
    scoring= 'neg_log_loss',
    cv=5)
gsearch.fit(X_train,y_train)
gsearch.best_params_  

{'learning_rate': 0.25}

In [ ]:
seed = 1234

rf_classifier = RandomForestClassifier(
                      min_samples_leaf=50,
                      n_estimators=150,
                      bootstrap=True,
                      oob_score=True,
                      n_jobs=-1,
                      random_state=seed,
                      max_features='auto')

In [18]:
param_test1 = {
    'min_samples_leaf' :[10, 20, 30, 50, 100, 150, 250]
}

gsearch = GridSearchCV(
    estimator = RandomForestClassifier(), 
    param_grid = param_test1, 
    scoring= 'neg_log_loss',
    cv=5)
gsearch.fit(X_train,y_train)
gsearch.best_params_  

{'min_samples_leaf': 20}

In [20]:
param_test2 = {
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}
gsearch = GridSearchCV(
    estimator = RandomForestClassifier(min_samples_leaf=20), 
    param_grid = param_test2, 
    scoring= 'neg_log_loss',
    cv=5)
gsearch.fit(X_train,y_train)
gsearch.best_params_     

{'n_estimators': 1800}

In [21]:
# tuning 'min_child_weight'
param_test3 = {
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]
}
gsearch3 = GridSearchCV(
    estimator = RandomForestClassifier(min_samples_leaf=20 , n_estimators = 1000), 
    param_grid = param_test3, 
    scoring= 'neg_log_loss',
    cv=5)
gsearch3.fit(X_train,y_train)
gsearch3.best_params_     

{'max_depth': 60}

In [22]:
#dtrain = RandomForestClassifier.DMatrix(data=X_train,label=y_train)
#dval = RandomForestClassifier.DMatrix(data=X_test,label=y_test)

param = { 'min_samples_leaf':20 , 'n_estimators': 1000, 'max_depth':60}
num_round = 1500
evallist = [(dval, 'eval'), (dtrain, 'train')]
bst = RandomForestClassifier.train(param, dtrain, num_round, evallist,early_stopping_rounds=80)

AttributeError: type object 'RandomForestClassifier' has no attribute 'DMatrix'

In [23]:
ypredval = bst.predict(dval, ntree_limit=bst.best_ntree_limit)

c:\Users\jacks\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [26]:
sklearn.metrics.log_loss(y_test, ypredval, normalize=True, sample_weight=None, labels=None)

0.4290323089484148